# Advection Diffusion Solver 


## Problem statement

We are investigating the Advection Diffusion Equation
$$
  \frac{\partial u^{\mathrm{MS}}}{\partial t} +\vec{u} \cdot \nabla u^{\mathrm{MS}} - \mu \nabla^2 u^{\mathrm{MS}} + S(u)=0 \in (0,2)^2
$$
with the initial value
$$
    u_0(x,y) = \sin(a x) \sin(b y),
$$
and the manufactured solution
$$
u^{\mathrm{MS}} = \sin(a x) \sin(b y) \cos(c t)
$$

#### Calculation of the Source Term
$$ 
\frac{\partial u^{\mathrm{MS}}}{\partial t} = -c \sin(a x) \sin(b y) \sin(c t)
$$
$$
\vec{u} \cdot \nabla u^{\mathrm{MS}} = \cos(ct) [u_1 a \cos(ax) \sin(by)+ u_2 b \sin(ax) \cos(by)]
$$
$$
\mu \nabla^2 u^{\mathrm{MS}} = \mu \cos(ct)[-a^2 \sin(ax) \sin(by)  - b^2 \sin(ax) \sin(by)]
$$

Full
$$
S(u) = -(\frac{\partial u^{\mathrm{MS}}}{\partial t} +\vec{u} \cdot \nabla u^{\mathrm{MS}} - \mu \nabla^2 u^{\mathrm{MS}}) 
$$


In [16]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Databases loaded: 
Capacity: 0
Count: 0



Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 204
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#17.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [17]:
using BoSSS.Solution.XdgTimestepping;

## Creation of Grid and Differential Operator

Here, 1 2D, $16 \times 16$ mesh is used:

In [18]:
var nCells=16;
var grid = Grid2D.Cartesian2DGrid(GenericBlas.Linspace(0.0,2.0,nCells+1), GenericBlas.Linspace(0.0,2.0,nCells+1));

Regarding the spatial part of the operator, we re-use the SIP-implementation 
which is already available in the `BoSSS.Solution.NSECommon` library.
For this implementation, whe have to specify the diffusion coefficient (here to -0.5)
as well as the Location of the Dirichlet boundary.

In [23]:
class Laplace : BoSSS.Solution.NSECommon.SIPLaplace {

    public Laplace() : base(1.4, "u") { } // override the constuctor; 
    //                                       The factor 1.4 is a constant "safty factor" multiper for the penalty parameter.
    //                                       The base implementation takes care about all other penalty factor dependencies, 
    //                                       i.e. dependence of the penatly factor on local cell size, DG polynomial degree and cell shape.
    //                                       The second argument, "u", is the name of the variable for the `ArgumentOrdering`.


    // Specifies whetherss a specific point (`inp.X`) is either a Dirichlet or a Neumann boundary;
    // Since in our case ther, the entire boundary should be Diriclet, we always return true.
    protected override bool IsDirichlet(ref CommonParamsBnd inp) {
        return true;
    }
    
    // diffusion coefficient
    override public double Nu(double[] x, double[] p, int jCell) {
        return -0.5;
    }

}

#### Advection Operator

First, we define functions for the given velocity field and the exact solution

In [39]:
public static class MyGlobals {
    public static double u(double[] X) => 0.8;
    public static double v(double[] X) => -0.6;
    public static double a = 2.3;
    public static double b = 2.9;
    public static double c = 4.2;
    public static double Nuu=0.5;

    public static double uExact(double[] X, double t) => Math.Sin(a * X[0]) * Math.Sin(b * X[1]) * Math.Cos(c * t);

    public static double TimeDerivative(double[] X, double t) => -c * Math.Sin(a * X[0]) * Math.Sin(b * X[1]) * Math.Sin(c * t);

    public static double ConvectiveTerm(double[] X, double t) => 
    Math.Cos(c * t) * (u(X) * a * Math.Cos(a * X[0]) * Math.Sin(b * X[1]) + v(X) * b * Math.Sin(a * X[0]) * Math.Cos(b * X[1]));

    public static double DiffusionTerm(double[] X, double t) => nu * Math.Cos(c * t) * (-a*a * Math.Sin(a * X[0]) * Math.Sin(b * X[1]) - b*b * Math.Sin(a * X[0]) * Math.Sin(b * X[1]));

    // Calculate the corresponding source term S(u)
    public static double S_u (double[] X, double t) =>  -(TimeDerivative(X,t)+ConvectiveTerm(X,t)-DiffusionTerm(X,t));
}

The flux we are considering is the Lax-Friedrichs flux
$$
\hat{f}(c_h^-, c_h^+, \vec{u} \cdot \vec{n}) = \overline{\vec{u} \, c_h} \cdot \vec{n} + \frac{C}{2} \lbrack \lbrack {c_h} \rbrack \rbrack.
$$
The constant $C \in \mathbb{R}^+$ has to be sufficiently large in order to guarantee the stability of the 
numerical scheme. We choose 
$$
C = \max \vert{\vec{u} \cdot \vec{n}}\vert = 1
$$
for the given problem.

In [21]:
double C = 0.3;     
 
Func<double, double, Vector, Vector, double> laxFriedrichsFlux =     
    delegate(double Uin, double Uout, Vector n, Vector velocityVector) {     
 
        return 0.5 * (Uin + Uout) * velocityVector * n - C * (Uout - Uin);     
    };

In [22]:

class ScalarTransportFlux : NonlinearFlux {     
 
    private Func<double, double, Vector, Vector, double> numericalFlux;     
 
    // Provides instances of this class with a specific flux implementation     
    public ScalarTransportFlux(     
        Func<double, double, Vector, Vector, double> numericalFlux) {     
 
        this.numericalFlux = numericalFlux;     
    }     
 
    public override IList<string> ArgumentOrdering {     
        get { return new string[] { "u" }; }     
    }     
 
    protected override void Flux(     
        double time, double[] x, double[] U, double[] output) {     
 
        output[0] = MyGlobals.u(x) * U[0];     
        output[1] = MyGlobals.v(x) * U[0];     
    }     
 
    // Makes use of the flux implementation supplied in the constructor     
    protected override double InnerEdgeFlux(     
        double time, double[] x, double[] normal,      
        double[] Uin, double[] Uout, int jEdge) {     
 
        Vector n              = new Vector(normal);     
        Vector velocityVector = new Vector(MyGlobals.u(x), MyGlobals.v(x));     
 
        return numericalFlux(Uin[0], Uout[0], n, velocityVector);     
    }     
 
    protected override double BorderEdgeFlux(     
        double time, double[] x, double[] normal, byte EdgeTag,     
        double[] Uin, int jEdge) {     
 
        double[] Uout = new double[] { MyGlobals.cExact(x, time) };     
 
        return InnerEdgeFlux(time, x, normal, Uin, Uout, jEdge);     
    }     
}

In [29]:
class SourceTerm : IVolumeForm
{

    public TermActivationFlags VolTerms => TermActivationFlags.UxV;

    public IList<string> ArgumentOrdering {     
        get { return new string[] { "u" }; }     
    } 

    public IList<string> ParameterOrdering => null;

    public double VolumeForm(ref CommonParamsVol cpv, double[] U, double[,] GradU, double V, double[] GradV)
    {
        //return U[0]*V;
        return MyGlobals.S_u(U,cpv.time)*V;
    }
}

#### Operator assembly
We compose the differential operator from the previously created 
`Laplace` implementation and add the `ScalarTransportFlux`. 

In [27]:
var Op = new DifferentialOperator(1, 0, 1, QuadOrderFunc.Linear(), "u", "R1");

Op.EquationComponents["R1"].Add(new Laplace()); // adding Laplace
Op.EquationComponents["R1"].Add(new ScalarTransportFlux(laxFriedrichsFlux)); // adding Advection
Op.EquationComponents["R1"].Add(new SourceTerm()); // adding Advection
Op.TemporalOperator = new ConstantTemporalOperator(Op);
Op.IsLinear = true;
Op.Commit();

## Find the time step

## Performing Temporal Integration

In [32]:

SinglePhaseField TimeIntegrate(SinglePhaseField u0, double EndTime, double dt, TimeSteppingScheme tsScheme=TimeSteppingScheme.SDIRK_54) {
    var u1 = u0.CloneAs();
    var Timestepper = new XdgTimestepping(Op, new DGField[] { u1 }, new DGField[] { new SinglePhaseField(u0.Basis)}, 
                            tsScheme);

    int NoOfTimeteps = (int) (EndTime/dt);
    for(int i = 0; i < NoOfTimeteps; i++)
        Timestepper.Solve(dt*i, dt);
    
    return u1;
}

Specify the initial value; For the initial field we also specify the DG polynomial degree 4,
which also sets the DG polynomial degree for the numerical solution.

In [33]:
var u0 = new SinglePhaseField(new Basis(grid, 4), "u");
u0.ProjectField((double[] X) => Math.Cos(X[0]*Math.PI*0.5)*Math.Cos(X[1]*Math.PI*0.5));

In [34]:
Tecplot("u0", u0);

Supersampling 4 requested, but limiting to 3 because higher values would very likely exceed this computers memory.
Note: Higher supersampling values are supported by external plot application, or by using e.g. the Tecplot class directly.
Writing output file c:\experimental\public\doc\handbook\AdvectionDiffusionSolver\u0...
done.


In [35]:
//for(int i = 0; i < 10; i++) {
//    Console.WriteLine("   --------------  timestep: " + (i + 1));
//    var u1 = TimeIntegrate(u0, 0.1, 1);
//    Tecplot("u" + (i+1), u1);
//    u0 = u1;
//}

## Performing a temporal convergence study

In [36]:
var dtS = new double[] {0.01,0.005};
var solutions = new List<SinglePhaseField>();
for(int i = 0; i < dtS.Length; i++) {
    Console.WriteLine("   --------------  computing dt: " + dtS[i]);
    var u1 = TimeIntegrate(u0, 0.1, dtS[i]);
    //Tecplot("u1." + i, u1);
    solutions.Add(u1);
}

   --------------  computing dt: 0.01
   --------------  computing dt: 0.005


In [ ]:
for(int i = 0; i < solutions.Count; i++)
    Tecplot("u1." + i, solutions[i]);

Computing the error against the exact solution:

In [ ]:
double uExEq(double[] X) {
   double Lambda = -Math.PI.Pow2()*0.5;
   double tend = 0.1;
   double u0 = Math.Cos(X[0]*Math.PI*0.5)*Math.Cos(X[1]*Math.PI*0.5);
   return Math.Exp(Lambda*tend)*u0;
}

In [ ]:
double[] ErrEx = new double[solutions.Count];
for(int i = 0; i < ErrEx.Length; i++) {
    ErrEx[i] = solutions[i].L2Error(uExEq);
}

In [ ]:
ErrEx

Computing the error against the finest solution:

In [ ]:
var uFinest = solutions.Last();

In [ ]:
var Errs = solutions.Take(solutions.Count - 1).Select(u => u.L2Error(uFinest)).ToArray();

In [ ]:
Errs

Computing the error between each solution and the finer one:

In [ ]:
double[] diffs = new double[solutions.Count - 1];
for(int i = 0; i < diffs.Length; i++) {
    diffs[i] = solutions[i].L2Error(solutions[i + 1]);
}

In [ ]:
diffs

Computing the experimental error of the DG discretization with DG polynomial degree $k$ against degree $k-1$.

In [ ]:
double ErrHi(SinglePhaseField u) {
    SinglePhaseField uLo = new SinglePhaseField(new Basis(u.Basis.GridDat, u.Basis.Degree - 1));
    uLo.AccLaidBack(1.0, u);
    var uHi = u.CloneAs();
    uHi.AccLaidBack(-1.0, uLo);
    return uHi.L2Norm();
}

In [ ]:
double[] HiDeg = new double[solutions.Count];
for(int i = 0; i < HiDeg.Length; i++) {
    HiDeg[i] = ErrHi(solutions[i]);
}

In [ ]:
HiDeg

### Plotting different errors

In [ ]:
var gp = new Gnuplot();
double[] ts = NoOfTimestepS.Take(NoOfTimestepS.Length).Select(x => (double)x).ToArray();
double[] ts1 = ts.Take(NoOfTimestepS.Length - 1).ToArray();
gp.PlotLogXLogY(ts1, Errs,  title:"Exp Error", format:new PlotFormat("-xk"));
gp.PlotLogXLogY(ts1, diffs, title:"Diffs", format:new PlotFormat("-ob"));
gp.PlotLogXLogY(ts,  ErrEx, title:"Exact Error", format:new PlotFormat("-+r"));
gp.PlotLogXLogY(ts,  HiDeg, title:"High Degree Norm", format:new PlotFormat(":ok"));
gp.PlotNow()

Confirming the order of the time integration:

In [ ]:
double slope = ts.Take(6).LogLogRegressionSlope(Errs.Take(6));
slope

In [ ]:
NUnit.Framework.Assert.LessOrEqual(slope, -3.5, "time convergence of the DIRK_54 method is not reached.")

Note that the lower limit (lower plateau) of the error plots looks different, depending 
- whether we compare against the finest solution (aka. experimental error, "Exp Error" in plot),
- or we compare against the "real" exact solution ("Exact Error" in plot).

Fro finer time-steps, the experimental error is reducing further and further,
the lower plateau is the floating-point accuracy limit of the `double` data type.
The exact error hits its lower plateau at a much higher level,
where the accuracy limit by the DG polynomial degree and the spatial accuracy is reached.
I.e., using a finer resolution and/or higher DG degree will bring the lower limit of the red curve further down. 
The blue and red curve are not affected by the spatial resolution - one would have to 
use increased floating-point accuracy (not feasible for BoSSS) to get this limit lower.